In [1]:
import csv  # Importa el módulo csv
import pandas as pd
from Libreria.cargar_configbd import cargar_configbd
from Libreria.cargar_mes_a_procesar import cargar_mes_a_procesar

In [2]:
# Crear un cursor para ejecutar consultas
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [21]:
mes, año = cargar_mes_a_procesar.leer_csv_en_lista('mes_a_procesar.csv')
mes = 'mayo'
año = 2024
print(f"{mes}{año}")

mayo2024


In [23]:
#----------------------------TRIPSSEPT2023

In [6]:
# Crear tabla para insertar TRIPSSEPT
consultas_sql = [f"DROP TABLE IF EXISTS trips{mes}{año};",
    f"""CREATE TABLE trips{mes}{año} (
	tripid INTEGER,
    uid VARCHAR(50),
    departure TIMESTAMP,
    arrival TIMESTAMP,
    travel_time float,
    travel_distance float
);
    """]

for consulta_sql in consultas_sql:
    cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()

In [10]:
# Path al archivo
trips = f"Datos/UDA/trips{mes}{año}.csv"

# Read CSV viajes
df_trips = pd.read_csv(trips, quoting=csv.QUOTE_NONNUMERIC, sep=',', low_memory=False)

In [11]:
# Formatear fechas arrival y departure
df_trips['departure'] = df_trips['departure'].str.split('.', expand=True).iloc[:, 0].str.split('Z', expand=True).iloc[:, 0].str.replace('T', ' ', regex=False)
df_trips['arrival'] = df_trips['arrival'].str.split('.', expand=True).iloc[:, 0].str.split('Z', expand=True).iloc[:, 0].str.replace('T', ' ', regex=False)

In [12]:
# Tabla para insertar registros
nombre_tabla = f"trips{mes}{año}"
coumnas_tabla = ['tripid', 'uid', 'departure', 'arrival', 'travel_time', 'travel_distance']
marcadores = ', '.join(['%s' for _ in range(len(coumnas_tabla))])


# Recorre el DataFrame y inserta cada fila en la tabla
print(f"n_puntos: {len(df_trips)}")
prog = 0
for index, row in df_trips.iterrows():
    prog += 1
    print(f"Progreso: {prog}", end="\r")
    #Construye la consulta SQL con parámetros
    consulta_sql = f"INSERT INTO {nombre_tabla} VALUES ({marcadores})"
    # Ejecuta la consulta SQL con los valores de la fila actual
    cur.execute(consulta_sql, tuple(row[coumnas_tabla]))
    
# Confirma los cambios en la base de datos
conn.commit()

n_puntos: 48576


In [ ]:
#----------------------------PUNTOS

In [13]:
# Crear tabla para insertar TRIPSSEPT
consultas_sql = [f"DROP TABLE IF EXISTS puntos{mes}{año};",
    f"""CREATE TABLE IF NOT EXISTS puntos{mes}{año} (
    company_token VARCHAR(255),
    latitude FLOAT,
    longitude FLOAT,
    altitude FLOAT,
    recorded_at TIMESTAMP
    );
    """]

for consulta_sql in consultas_sql:
    cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()

In [16]:
# Path al archivo
points = f"Datos/UDA/puntos{mes}{año}.csv"

# Lee el archivo CSV con Pandas
df_points = pd.read_csv(points, quoting=csv.QUOTE_NONNUMERIC, sep=',', low_memory=False)


In [17]:
# Dividir la columna 'data' en varias columnas
df_points['recorded_at'] = df_points['recorded_at'].str.split('.', expand=True).iloc[:, 0].str.split('Z', expand=True).iloc[:, 0].str.replace('T', ' ', regex=False)

In [18]:

# Tabla para insertar registros
nombre_tabla = f"puntos{mes}{año}"
columnas_tabla = ['company_token', 'latitude', 'longitude', 'altitude', 'recorded_at']
marcadores = ', '.join(['%s' for _ in range(len(columnas_tabla))])

# Recorre el DataFrame y inserta cada fila en la tabla
print(f"n_puntos: {len(df_points)}")
prog = 0
for index, row in df_points.iterrows():
    prog += 1
    print(f"Progreso: {prog}", end="\r")
    #Construye la consulta SQL con parámetros
    consulta_sql = f"INSERT INTO {nombre_tabla} VALUES ({marcadores})"
    # Ejecuta la consulta SQL con los valores de la fila actual
    cur.execute(consulta_sql, tuple(row[columnas_tabla]))

    conn.commit()

cur.execute('''UPDATE puntossept2023
SET recorded_at = DATE_TRUNC('second', recorded_at);''')
        
# Confirma los cambios en la base de datos
conn.commit()

n_puntos: 6143402


In [ ]:
# ----------- UNIR DATOS TRIPS Y PUNTOS

In [19]:
consulta_sql = f'''DROP TABLE if exists join_viajes_puntos_{mes}{año};
                CREATE TABLE IF NOT EXISTS join_viajes_puntos_{mes}{año} AS
                        SELECT v.tripid, v.uid, p.latitude, p.longitude, p.recorded_at
                        FROM trips{mes}{año} v
                        JOIN puntos{mes}{año} p ON v.uid = p.company_token
                        WHERE p.recorded_at BETWEEN v.departure AND v.arrival;
                '''

cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()